In [6]:
#!pip3 install twilio

In [7]:
import os
from twilio.rest import Client
from twilio_config import *
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime


# Armado de la URL

In [3]:
query = 'Cusco'
api_key = API_KEY_WAPI

url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
url_clima

'http://api.weatherapi.com/v1/forecast.json?key=XXXXXXXXXXXXXXXXXXXX&q=Cusco&days=1&aqi=no&alerts=no'

In [ ]:
response = requests.get(url_clima).json()

In [ ]:
response

In [5]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [6]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [7]:
len(response['forecast']['forecastday'][0]['hour'])

24

In [8]:
response['forecast']['forecastday'][0]['hour'][0]

{'time_epoch': 1702270800,
 'time': '2023-12-11 00:00',
 'temp_c': 10.7,
 'temp_f': 51.2,
 'is_day': 0,
 'condition': {'text': 'Patchy rain possible',
  'icon': '//cdn.weatherapi.com/weather/64x64/night/176.png',
  'code': 1063},
 'wind_mph': 2.5,
 'wind_kph': 4.0,
 'wind_degree': 25,
 'wind_dir': 'NNE',
 'pressure_mb': 1016.0,
 'pressure_in': 30.0,
 'precip_mm': 0.07,
 'precip_in': 0.0,
 'humidity': 77,
 'cloud': 70,
 'feelslike_c': 10.9,
 'feelslike_f': 51.5,
 'windchill_c': 10.9,
 'windchill_f': 51.5,
 'heatindex_c': 10.7,
 'heatindex_f': 51.2,
 'dewpoint_c': 6.7,
 'dewpoint_f': 44.1,
 'will_it_rain': 1,
 'chance_of_rain': 71,
 'will_it_snow': 0,
 'chance_of_snow': 0,
 'vis_km': 10.0,
 'vis_miles': 6.0,
 'gust_mph': 2.9,
 'gust_kph': 4.7,
 'uv': 1.0}

In [9]:
response['forecast']['forecastday'][0]['hour'][0]['time'].split()[0] #Fecha

'2023-12-11'

In [10]:
response['forecast']['forecastday'][0]['hour'][0]['time'].split()[1].split(':')[0] #Hora

'00'

In [11]:
response['forecast']['forecastday'][0]['hour'][0]['condition']['text'] #Condition

'Patchy rain possible'

In [12]:
response['forecast']['forecastday'][0]['hour'][0]['temp_c'] #Temperatura

10.7

In [13]:
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain'] 

1

In [14]:
response['forecast']['forecastday'][0]['hour'][0]['chance_of_rain']

71

# Dataframe

In [15]:
def get_forecast(response,i):
    
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = float(response['forecast']['forecastday'][0]['hour'][i]['temp_c'])
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha,hora,condicion,tempe,rain,prob_rain

In [16]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):
    
    datos.append(get_forecast(response,i))
    

100%|██████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<?, ?it/s]


In [17]:
datos[0]

('2023-12-11', 0, 'Patchy rain possible', 10.7, 1, 71)

In [18]:
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','prob_lluvia']
df = pd.DataFrame(datos,columns=col)
df = df.sort_values(by = 'Hora',ascending = True)
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2023-12-11,0,Patchy rain possible,10.7,1,71
1,2023-12-11,1,Thundery outbreaks possible,10.5,1,87
2,2023-12-11,2,Thundery outbreaks possible,10.4,1,87
3,2023-12-11,3,Thundery outbreaks possible,10.2,1,77
4,2023-12-11,4,Thundery outbreaks possible,9.0,0,70
5,2023-12-11,5,Patchy rain possible,8.4,1,77
6,2023-12-11,6,Patchy rain possible,9.5,1,75
7,2023-12-11,7,Patchy rain possible,11.6,0,65
8,2023-12-11,8,Patchy light rain with thunder,13.0,1,94
9,2023-12-11,9,Patchy light rain with thunder,14.7,1,73


In [19]:
# data_cli = {
#     'Fecha':datos[0][0],
#     'Hora':datos[1][],
#     'Tiempo':condicion,
#     'Temperatura':tempe,
#     'LLoverá':rain,
#     'prob_lluvia':prob_rain
    
# }

# df = pd.DataFrame(data_cli)
# df = df.sort_values(by = 'Hora',ascending = True)

In [20]:
df[df['Lluvia']==1]

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2023-12-11,0,Patchy rain possible,10.7,1,71
1,2023-12-11,1,Thundery outbreaks possible,10.5,1,87
2,2023-12-11,2,Thundery outbreaks possible,10.4,1,87
3,2023-12-11,3,Thundery outbreaks possible,10.2,1,77
5,2023-12-11,5,Patchy rain possible,8.4,1,77
6,2023-12-11,6,Patchy rain possible,9.5,1,75
8,2023-12-11,8,Patchy light rain with thunder,13.0,1,94
9,2023-12-11,9,Patchy light rain with thunder,14.7,1,73
10,2023-12-11,10,Patchy rain possible,16.0,1,73
12,2023-12-11,12,Patchy rain possible,17.3,1,98


In [34]:
df_rain =  df[(df['Lluvia']==1) & (df['Hora']>6) & (df['Hora']< 22)]
df_rain = df_rain[['Hora','Condicion']]
df_rain.set_index('Hora', inplace = True)

In [35]:
df['Fecha'][0]

'2023-12-11'

In [36]:
df_rain

,Condicion
Hora,
8,Patchy light rain with thunder
9,Patchy light rain with thunder
10,Patchy rain possible
12,Patchy rain possible
13,Thundery outbreaks possible
15,Patchy rain possible
16,Patchy rain possible
17,Patchy rain possible
18,Patchy rain possible


In [37]:
'\nHola! \n\n\n El pronostico del tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain)

'\nHola! \n\n\n El pronostico del tiempo hoy 2023-12-11 en Cusco es : \n\n\n                            Condicion\nHora                                \n8     Patchy light rain with thunder\n9     Patchy light rain with thunder\n10              Patchy rain possible\n12              Patchy rain possible\n13       Thundery outbreaks possible\n15              Patchy rain possible\n16              Patchy rain possible\n17              Patchy rain possible\n18              Patchy rain possible\n20                     Partly cloudy\n21    Patchy light rain with thunder'

In [38]:
PHONE_NUMBER

'+1 510 356 2265'

# Mensaje SMS desde Twilio

In [39]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID 
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                     body='\nHola Choto! \n\n\n El pronostico de lluvia hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain),
                     from_=PHONE_NUMBER,
                     to='+51944242739'
                 )

print('Mensaje Enviado ' + message.sid)

Mensaje Enviado SMb9474114938c6662ef1734c70225a768
